In [1]:
import sys
import os
current_dir = os.getcwd()  #use to import the defined functions
parent_dir = os.path.dirname(current_dir) 
sys.path.append(parent_dir)  

"""
incase the above code does not work, you can use the absolute path instead
sys.path.append(r".\")
"""


'\nincase the above code does not work, you can use the absolute path instead\nsys.path.append(r".")\n'

In [2]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau 

In [3]:
from utils.basic_utils import setup_seed
from data.SimulationData import DataGenerator 
from utils.training_utils import train_WGR_fnn
from utils.evaluation_utils import L1L2_MSE_mean_sd_G, MSE_quantile_G_uniY
from models.generator import generator_fnn
from models.discriminator import discriminator_fnn

In [4]:
import argparse

if 'ipykernel_launcher.py' in sys.argv[0]:  #if not work in jupyter, you can delete this part
    import sys
    sys.argv = [sys.argv[0]] 


parser = argparse.ArgumentParser(description='Implementation of WGR for M1')

parser.add_argument('--Xdim', default=5, type=int, help='dimensionality of X')
parser.add_argument('--Ydim', default=1, type=int, help='dimensionality of Y')
parser.add_argument('--model', default='M2', type=str, help='model')

parser.add_argument('--noise_dim', default=5, type=int, help='dimensionality of noise vector')
parser.add_argument('--noise_dist', default='gaussian', type=str, help='distribution of noise vector')

parser.add_argument('--train', default=5000, type=int, help='size of train dataset')
parser.add_argument('--val', default=1000, type=int, help='size of validation dataset')
parser.add_argument('--test', default=1000, type=int, help='size of test dataset')

parser.add_argument('--train_batch', default=128, type=int, metavar='BS', help='batch size while training')
parser.add_argument('--val_batch', default=100, type=int, metavar='BS', help='batch size while validation')
parser.add_argument('--test_batch', default=100, type=int, metavar='BS', help='batch size while testing')
parser.add_argument('--epochs', default=100, type=int, help='number of epochs to train')
parser.add_argument('--reps', default=100, type=int, help='number of replications')

args = parser.parse_args()

print(args)

Namespace(Xdim=5, Ydim=1, model='M2', noise_dim=5, noise_dist='gaussian', train=5000, val=1000, test=1000, train_batch=128, val_batch=100, test_batch=100, epochs=100, reps=100)


In [5]:
# Set seed 
setup_seed(5678)

In [6]:
# Generate data from M2
data_gen = DataGenerator(args)
DATA = data_gen.generate_data('M2')
train_X, train_Y = DATA['train_X'], DATA['train_Y']
val_X, val_Y = DATA['val_X'], DATA['val_Y']
test_X, test_Y = DATA['test_X'], DATA['test_Y']

In [7]:
# Create TensorDatasets and initialize a DataLoaders
train_dataset = TensorDataset( train_X.float(), train_Y.float() )
loader_train = DataLoader(train_dataset , batch_size=args.train_batch, shuffle=True)

val_dataset = TensorDataset( val_X.float(), val_Y.float() )
loader_val = DataLoader(val_dataset , batch_size=args.val_batch, shuffle=True)

test_dataset = TensorDataset( test_X.float(), test_Y.float() )
loader_test  = DataLoader(test_dataset , batch_size=args.test_batch, shuffle=True)

In [8]:
# Define generator network and discriminator network
G_net = generator_fnn(Xdim=args.Xdim, Ydim=args.Ydim, noise_dim=args.noise_dim, hidden_dims = [64, 32])
D_net = discriminator_fnn(input_dim=args.Xdim+args.Ydim, hidden_dims = [64, 32])

In [9]:
# Initialize RMSprop optimizers
D_solver = optim.Adam(D_net.parameters(), lr=0.001, betas=(0.9, 0.999))
G_solver = optim.Adam(G_net.parameters(), lr=0.001, betas=(0.9, 0.999))                    


In [10]:
# Training
trained_G, trained_D = train_WGR_fnn(D=D_net, G=G_net, D_solver=D_solver, G_solver=G_solver, 
                                     loader_train = loader_train, loader_val=loader_val,
                                     noise_dim=args.noise_dim, Xdim=args.Xdim, Ydim=args.Ydim, batch_size=args.train_batch,
                                     save_path='./', model_type=args.model, device='cpu', num_epochs=200)

Mean L1 Loss: 6.728624, Mean L2 Loss: 86.253746
Epoch 0 - D Loss: 2.7911, G Loss: 7.5823
Epoch 1 - D Loss: -2.2032, G Loss: 1.6467
Epoch 2 - D Loss: 2.0292, G Loss: -1.9246
Epoch 3 - D Loss: 0.3748, G Loss: 4.5907
Epoch 4 - D Loss: 0.7547, G Loss: 6.9154
Epoch 5 - D Loss: 0.2695, G Loss: 4.0228
Epoch 6 - D Loss: 0.1154, G Loss: 1.6121
Epoch 7 - D Loss: -0.1679, G Loss: 4.6640
Epoch 8 - D Loss: 0.2876, G Loss: 4.2864
Epoch 9 - D Loss: -0.0555, G Loss: 2.2260
Epoch 10 - D Loss: -0.1342, G Loss: 2.6173
Epoch 11 - D Loss: 0.1297, G Loss: 2.3706
Epoch 12 - D Loss: 0.1163, G Loss: 0.4810
Epoch 13 - D Loss: 0.1085, G Loss: -0.0949
Epoch 14 - D Loss: 0.0586, G Loss: -0.0394
Epoch 15 - D Loss: 0.0377, G Loss: -0.1144
Epoch 16 - D Loss: 0.0252, G Loss: -0.0896
Epoch 17 - D Loss: 0.0132, G Loss: -0.2361
Epoch 18 - D Loss: 0.0025, G Loss: -0.3572
Epoch 19 - D Loss: -0.0075, G Loss: -0.3748
Epoch 20 - D Loss: -0.0210, G Loss: -0.4515
Epoch 21 - D Loss: -0.0322, G Loss: -0.3666
Epoch 22 - D Loss: -0

In [11]:
# Calculate the L1 and L2 error, MSE of conditional mean and conditional standard deviation on the test data  
test_G_mean_sd = L1L2_MSE_mean_sd_G(G = trained_G,  test_size = args.test, noise_dim=args.noise_dim, Xdim=args.Xdim,
                                    batch_size=args.test_batch,  model_type=args.model, loader_dataset = loader_test )

Model: M2, Univariate, Ydim: 1, J_t_size: 50
L1 Loss: tensor([0.7289])
L2 Loss: tensor([0.8540])
MSE Mean: tensor([0.0745])
MSE SD: tensor([0.0662])


In [12]:
# Calculate the MSE of conditional quantiles at different levels.
test_G_quantile = MSE_quantile_G_uniY(G = trained_G, loader_dataset = loader_test , noise_dim=args.noise_dim, Xdim=args.Xdim,
                                      test_size = args.test,  batch_size=args.test_batch, model_type=args.model)

Q_5: 0.6683, Q_25: 0.0968, Q_50: 0.1975, Q_75: 0.1257, Q_95: 0.4440
